#**Обработка таблиц по критериям заказчика, исключение выбросов, выделение записей классов целевой / нецелевой.**

##Павелецкая_(pavcity.turbo.site).

In [ ]:
import pandas as pd
import numpy as np
import datetime
import zipfile
import os
import re

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи звонков_павелецкая сити-20231022T075749Z-001.zip'

In [ ]:
df_Pavcity_turbo_site = pd.read_excel('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты необработанные/Павелецкая (pavcity.turbo.site).xlsx')
df_Pavcity_turbo_site.to_csv('Павелецкая_(pavcity.turbo.site)_(Необработанная).csv', index = False)
path = '/content/Павелецкая_(pavcity.turbo.site)_(Необработанная).csv'
df_Pavcity_turbo_site = pd.read_csv(path)

In [ ]:
df_Pavcity_turbo_site

,Статус,Тип,Дата и время,Номер абонента,Длительность звонка,Длительность ожидания ответа,Сотрудник,ID посетителя,Номер обращения,Теги,...,Чистая длительность разговора,Полная длительность ожидания,Идентификатор звонка во внешней системе,Расширенная UTM-метка Source,Расширенная UTM-метка Campaign,Расширенная UTM-метка Medium,Расширенная UTM-метка Content,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid
0,Потерянный,Динамический коллтрекинг,2023-07-27 14:53:43,79894365488,00:00:01,00:00:01,7.495514e+10,7.095819e+09,1,Не обработано - Нецелевой контакт,...,00:00:00,00:00:01,NaN,yandex,ya_paveletskaya_kz_n_geo,cpc,v2||64424254||11012716629||33325751517||жилые ...,жилые комплексы на юге москвы,NaN,NaN
1,Принятый,Динамический коллтрекинг,2023-07-26 12:57:19,375295142037,00:00:10,00:00:03,7.495514e+10,7.147024e+09,1,Нецелевой контакт,...,00:00:07,00:00:03,NaN,yandex,ya_paveletskaya_kz_n_geo,cpc,v2||64424254||13819492116||44050215064||---aut...,---autotargeting,NaN,NaN
2,Принятый,Динамический коллтрекинг,2023-07-20 19:14:10,79331115364,00:00:23,00:00:01,7.495514e+10,7.470856e+09,1,Нецелевой контакт,...,00:00:22,00:00:01,NaN,yandex,ya_paveletskaya_msk_r_retargeting,cpc,v2||50264576||11707219000||42082330||||0||none...,NaN,NaN,NaN
3,Принятый,Динамический коллтрекинг,2023-07-18 14:54:11,79680573141,00:01:00,00:00:01,7.495514e+10,7.107143e+09,1,NaN,...,00:00:59,00:00:01,NaN,yandex,ya_paveletskaya_kz_n_geo,cpc,v2||64424254||13819492123||44050215064||---aut...,---autotargeting,NaN,NaN
4,Принятый,Динамический коллтрекинг,2023-07-18 13:51:58,78005055934,00:00:02,00:00:01,7.495514e+10,7.046435e+09,2,Нецелевой контакт,...,00:00:01,00:00:01,NaN,yandex,ya_paveletskaya_kz_n_geo,cpc,v2||64424254||11012716629||33325751427||купить...,купить двушку на юге москвы,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,Принятый,Динамический коллтрекинг,2023-05-03 16:43:18,79373021536,00:00:14,00:00:02,7.495514e+10,7.097844e+09,1,Нецелевой контакт,...,00:00:12,00:00:02,NaN,yandex,ya_paveletskaya_msk_r_retargeting,cpc,v2||50264576||11707218989||42082330||||0||none...,NaN,NaN,NaN
1207,Потерянный,Динамический коллтрекинг,2023-05-03 06:28:01,77058938377,00:00:00,00:00:00,NaN,7.096342e+09,2,Не обработано - Нецелевой контакт,...,00:00:00,00:00:00,NaN,yandex,ya_paveletskaya_kz_n_geo,cpc,v2||64424254||11012716629||44050215065||---aut...,---autotargeting,NaN,NaN
1208,Принятый,Динамический коллтрекинг,2023-05-03 06:27:25,77058938377,00:00:14,00:00:03,7.495514e+10,7.096342e+09,1,Нецелевой контакт,...,00:00:11,00:00:03,NaN,yandex,ya_paveletskaya_kz_n_geo,cpc,v2||64424254||11012716629||44050215065||---aut...,---autotargeting,NaN,NaN
1209,Принятый,Динамический коллтрекинг,2023-05-02 19:52:09,77779012504,00:01:00,00:00:02,7.495514e+10,7.071742e+09,1,NaN,...,00:00:58,00:00:02,NaN,yandex,ya_paveletskaya_kz_n_geo,cpc,v2||64424254||13819492115||44050214991||компле...,комплекс москва москва,NaN,NaN


In [ ]:
df_Pavcity_turbo_site.Теги.unique()

array(['Не обработано - Нецелевой контакт', 'Нецелевой контакт', nan,
       'Обработано - Нецелевой контакт'], dtype=object)

In [ ]:
df_Pavcity_turbo_site = df_Pavcity_turbo_site.dropna(subset = ['Теги'])
df_Pavcity_turbo_site = df_Pavcity_turbo_site.dropna(subset = ['Чистая длительность разговора'])

In [ ]:
df_Pavcity_turbo_site.loc[~df_Pavcity_turbo_site['Теги'].str.contains('Целевой_М108'), 'Теги'] = 'Нецелевой'

In [ ]:
df_Pavcity_turbo_site['Теги'] = df_Pavcity_turbo_site['Теги'].astype(str)
df_Pavcity_turbo_site['Теги'] = df_Pavcity_turbo_site['Теги'].apply(lambda x: 'Нецелевой' if 'Нецелевой' in x else x)

In [ ]:
audio_folder = '/content/Записи звонков_павелецкая сити'
audio_files = os.listdir(audio_folder)
session_ids = [re.findall(r'session_(\d+)', file)[0] for file in audio_files]
unique_ids = df_Pavcity_turbo_site['Идентификатор сессии звонка'].unique()
unique_ids_in_files = [id for id in unique_ids if str(id) in session_ids]
df_Pavcity_turbo_site = df_Pavcity_turbo_site[df_Pavcity_turbo_site['Идентификатор сессии звонка'].isin(unique_ids_in_files)]

In [ ]:
df_Pavcity_turbo_site['Чистая длительность разговора'] = pd.to_timedelta(df_Pavcity_turbo_site['Чистая длительность разговора'])

In [ ]:
df_Pavcity_turbo_site = df_Pavcity_turbo_site.reset_index(drop = True)

In [ ]:
df_Pavcity_turbo_site

,Статус,Тип,Дата и время,Номер абонента,Длительность звонка,Длительность ожидания ответа,Сотрудник,ID посетителя,Номер обращения,Теги,...,Чистая длительность разговора,Полная длительность ожидания,Идентификатор звонка во внешней системе,Расширенная UTM-метка Source,Расширенная UTM-метка Campaign,Расширенная UTM-метка Medium,Расширенная UTM-метка Content,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid
0,Принятый,Динамический коллтрекинг,2023-07-18 13:51:58,78005055934,00:00:02,00:00:01,7.495514e+10,7.046435e+09,2,Нецелевой,...,0 days 00:00:01,00:00:01,NaN,yandex,ya_paveletskaya_kz_n_geo,cpc,v2||64424254||11012716629||33325751427||купить...,купить двушку на юге москвы,NaN,NaN
1,Принятый,Динамический коллтрекинг,2023-07-11 18:13:13,79274147009,00:00:04,00:00:01,7.495514e+10,7.098722e+09,2,Нецелевой,...,0 days 00:00:03,00:00:01,NaN,yandex,ya_paveletskaya_kz_n_geo,cpc,v2||64424254||11012716629||44050215065||---aut...,---autotargeting,NaN,NaN
2,Принятый,Аналитика,2023-06-27 15:04:42,79036175926,00:00:22,00:00:02,7.495514e+10,NaN,1,Нецелевой,...,0 days 00:00:20,00:00:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Принятый,Динамический коллтрекинг,2023-06-19 20:04:38,79274574090,00:00:04,00:00:01,7.495514e+10,7.371932e+09,1,Нецелевой,...,0 days 00:00:02,00:00:02,NaN,yandex,ya_paveletskaya_kz_n_geo,cpc,v2||64424254||11012716635||44050215065||---aut...,---autotargeting,NaN,NaN


In [ ]:
df_Pavcity_turbo_site.Теги.unique()

array(['Нецелевой'], dtype=object)

In [ ]:
counts = df_Pavcity_turbo_site['Теги'].value_counts()
print(counts)

Нецелевой    4
Name: Теги, dtype: int64


In [ ]:
df_Pavcity_turbo_site.to_csv('df_Pavcity_turbo_site.csv', index = False)

##Павелецкая_ЖК_«Павелецкая_сити».

In [ ]:
import pandas as pd
import numpy as np
import datetime
import zipfile
import os
import re

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи звонков_павелецкая сити-20231022T075749Z-001.zip'

In [ ]:
df_Paveletskaya = pd.read_excel('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты необработанные/Павелецкая_ЖК_«Павелецкая_сити».xlsx')
df_Paveletskaya.to_csv('Павелецкая_ЖК_«Павелецкая_сити»_(Необработанная).csv', index = False)
path = '/content/Павелецкая_ЖК_«Павелецкая_сити»_(Необработанная).csv'
df_Paveletskaya = pd.read_csv(path)

In [ ]:
df_Paveletskaya

,Статус,Тип,Дата и время,Номер абонента,Длительность ожидания ответа,Сотрудник,Рекламная кампания,Номер обращения,Длительность звонка,Теги,...,Чистая длительность разговора,Полная длительность ожидания,Идентификатор звонка во внешней системе,Расширенная UTM-метка Source,Расширенная UTM-метка Campaign,Расширенная UTM-метка Medium,Расширенная UTM-метка Content,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid
0,Принятый,Аналитика,2023-07-20 11:43:08,79166458242,00:00:01,SIP URI,Media108 | Павелецкая Сити | cpa | Roomberry,1,00:07:09,Целевой_М108,...,00:07:08,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Принятый,Аналитика,2023-07-20 11:37:35,79112808417,00:00:02,74955141111,MRGroup | Павелецкая Сити | offline | Брендинг...,1,00:02:10,NaN,...,00:02:08,00:00:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Принятый,Аналитика,2023-07-20 11:37:10,79099624838,00:00:01,SIP URI,Media108 | Павелецкая Сити | xml | Яндекс.Недв...,1,00:01:21,NaN,...,00:01:20,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Принятый,Аналитика,2023-07-20 11:36:58,79169021186,00:00:01,SIP URI,Media108 | Павелецкая Сити | media | 2realty,1,00:00:38,NaN,...,00:00:37,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Принятый,Аналитика,2023-07-20 11:33:47,79168455509,00:00:01,74955141111,Media108 | Павелецкая Сити | direct | Google A...,17,00:03:51,NaN,...,00:03:50,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082,Принятый,Аналитика,2023-06-17 16:16:05,79680303488,00:00:01,SIP URI,Media108 | Павелецкая Сити | media | Novostroy-m,1,00:06:40,Целевой_М108,...,00:06:39,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1083,Принятый,Аналитика,2023-06-17 14:21:31,79259041087,00:00:00,SIP URI,Media108 | Павелецкая Сити | media | Avaho,2,00:05:46,NaN,...,00:05:46,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1084,Принятый,Аналитика,2023-06-17 14:08:50,74993205445,00:00:02,74955141111,MRGroup | Павелецкая Сити | seo | Яндекс Поиск,1,00:03:11,NaN,...,00:03:09,00:00:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1085,Принятый,Аналитика,2023-06-17 13:44:16,79066743768,00:00:01,SIP URI,Media108 | Павелецкая Сити | cpa | Roomberry,1,00:01:33,Целевой_М108,...,00:01:32,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_Paveletskaya.Теги.unique()

array(['Целевой_М108', nan, 'Нецелевой контакт',
       'Не обработано - Нецелевой контакт',
       'Нецелевой контакт - Целевой_М108', 'Нецелевой',
       'Обработано - Нецелевой контакт', 'Не обработано',
       'Коммерция_целевой_М108',
       'Обработано - Нецелевой контакт - Целевой_М108'], dtype=object)

In [ ]:
df_Paveletskaya = df_Paveletskaya.dropna(subset = ['Теги'])
df_Paveletskaya = df_Paveletskaya.dropna(subset = ['Чистая длительность разговора'])

In [ ]:
df_Paveletskaya.loc[~df_Paveletskaya['Теги'].str.contains('Целевой_М108', case=False), 'Теги'] = 'Нецелевой'

In [ ]:
df_Paveletskaya['Теги'] = df_Paveletskaya['Теги'].astype(str)
df_Paveletskaya['Теги'] = df_Paveletskaya['Теги'].apply(lambda x: 'Целевой' if 'целевой_м108' in x.lower() else x)
df_Paveletskaya['Теги'] = df_Paveletskaya['Теги'].apply(lambda x: 'Нецелевой' if 'Нецелевой' in x else x)

In [ ]:
audio_folder = '/content/Записи звонков_павелецкая сити'
audio_files = os.listdir(audio_folder)
session_ids = [re.findall(r'session_(\d+)', file)[0] for file in audio_files]
unique_ids = df_Paveletskaya['Идентификатор сессии звонка'].unique()
unique_ids_in_files = [id for id in unique_ids if str(id) in session_ids]
df_Paveletskaya = df_Paveletskaya[df_Paveletskaya['Идентификатор сессии звонка'].isin(unique_ids_in_files)]

In [ ]:
df_Paveletskaya['Чистая длительность разговора'] = pd.to_timedelta(df_Paveletskaya['Чистая длительность разговора'])
df_Paveletskaya = df_Paveletskaya[~((df_Paveletskaya['Чистая длительность разговора'] < datetime.timedelta(seconds = 75)) & (df_Paveletskaya['Теги'] == 'Целевой'))]

In [ ]:
targeted_rows = df_Paveletskaya[df_Paveletskaya['Теги'] == 'Целевой']
df_Targeted_Sample = targeted_rows.sample(frac=0.1, random_state=1)
nontargeted_rows = df_Paveletskaya[df_Paveletskaya['Теги'] == 'Нецелевой']
df_NonTargeted_Sample = nontargeted_rows.sample(frac=0.1, random_state=1)
df_Test_Pav1 = pd.concat([df_Targeted_Sample, df_NonTargeted_Sample])
df_Paveletskaya = df_Paveletskaya.drop(df_Test_Pav1.index)
df_Test_Pav1.to_csv('df_Test_Pav1.csv', index=False)

In [ ]:
df_Paveletskaya = df_Paveletskaya.reset_index(drop = True)

In [ ]:
df_Paveletskaya

,Статус,Тип,Дата и время,Номер абонента,Длительность ожидания ответа,Сотрудник,Рекламная кампания,Номер обращения,Длительность звонка,Теги,...,Чистая длительность разговора,Полная длительность ожидания,Идентификатор звонка во внешней системе,Расширенная UTM-метка Source,Расширенная UTM-метка Campaign,Расширенная UTM-метка Medium,Расширенная UTM-метка Content,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid
0,Принятый,Аналитика,2023-07-20 11:43:08,79166458242,00:00:01,SIP URI,Media108 | Павелецкая Сити | cpa | Roomberry,1,00:07:09,Целевой,...,0 days 00:07:08,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Принятый,Аналитика,2023-07-20 10:49:37,79634170796,00:00:02,74955141111,MRGroup | Павелецкая Сити | direct | Яндекс.Ди...,3,00:00:06,Нецелевой,...,0 days 00:00:04,00:00:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Принятый,Автоперезвон по заявкам,2023-07-20 10:39:51,79774804219,00:00:39,Обратный звонок (SIP<1),Media108 | Павелецкая Сити | direct | Яндекс.Д...,2,00:42:51,Целевой,...,0 days 00:42:12,00:00:39,NaN,yandex,m108_pavelcity_brand_search_mmo|87584869,cpc,id|44890473407_44890473407|cid|87584869|gid|52...,павелецкий сити,NaN,NaN
3,Принятый,Аналитика,2023-07-19 21:28:07,79099359199,00:00:01,74955141111,MRGroup | Павелецкая Сити | sms | СМС рассылка...,1,00:00:18,Нецелевой,...,0 days 00:00:17,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Принятый,Динамический коллтрекинг,2023-07-19 18:35:47,79261666111,00:00:01,74955141111,Media108 | Павелецкая Сити | direct | Яндекс.Д...,2,00:03:37,Целевой,...,0 days 00:03:36,00:00:01,NaN,yandex,brand_search_mmo|87584869,cpc,id|44890473306_44890473306|cid|87584869|gid|52...,жк павелецкая сити,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,Принятый,Аналитика,2023-06-18 10:07:09,79151900879,00:00:01,SIP URI,Media108 | Павелецкая Сити | media | Gdetodom,1,00:12:13,Целевой,...,0 days 00:12:12,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
443,Принятый,Аналитика,2023-06-18 09:22:51,79651805758,00:00:01,74955141111,MRGroup | Павелецкая Сити | offline | Брендиро...,1,00:00:25,Нецелевой,...,0 days 00:00:24,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
444,Принятый,Динамический коллтрекинг,2023-06-17 17:21:56,79094690999,00:00:02,74955141111,Media108 | Павелецкая Сити | direct | Яндекс.Д...,2,00:00:09,Нецелевой,...,0 days 00:00:07,00:00:02,NaN,yandex,competitors_general_call_rsy_mmo|88841292,cpc,id|45120041544_45120041544|cid|88841292|gid|52...,жк павелецкая сити мр групп,NaN,NaN
445,Принятый,Аналитика,2023-06-17 13:44:16,79066743768,00:00:01,SIP URI,Media108 | Павелецкая Сити | cpa | Roomberry,1,00:01:33,Целевой,...,0 days 00:01:32,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_Paveletskaya.Теги.unique()

array(['Целевой', 'Нецелевой'], dtype=object)

In [ ]:
counts = df_Paveletskaya['Теги'].value_counts()
print(counts)

Целевой      246
Нецелевой    201
Name: Теги, dtype: int64


In [ ]:
df_Paveletskaya.to_csv('df_Paveletskaya.csv', index = False)

##Дополнительный DF по Павелецкая_ЖК_«Павелецкая_сити».

In [ ]:
import pandas as pd
import numpy as np
import datetime
import zipfile
import os
import re

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи_Павелецкая_сити_дополнительные-20231028T005609Z-001.zip'

In [ ]:
df_Paveletskaya2 = pd.read_excel('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты необработанные/Павелецкая_сити_дополнительные.xlsx')
df_Paveletskaya2.to_csv('Павелецкая_ЖК_«Павелецкая_сити»_(Необработанная2).csv', index = False)
path = '/content/Павелецкая_ЖК_«Павелецкая_сити»_(Необработанная2).csv'
df_Paveletskaya2 = pd.read_csv(path)

In [ ]:
df_Paveletskaya2

,Статус,Тип,Дата и время,Номер абонента,Теги,Рекламная кампания,Длительность звонка,Client ID Яндекс.Метрика,Длительность ожидания ответа,Сотрудник,...,Чистая длительность разговора,Полная длительность ожидания,Идентификатор звонка во внешней системе,Расширенная UTM-метка Source,Расширенная UTM-метка Campaign,Расширенная UTM-метка Medium,Расширенная UTM-метка Content,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid
0,Принятый,Аналитика,2023-10-04 22:46:01,7.968536e+10,Нецелевой контакт,Media108 | Павелецкая Сити | catalog | Google ...,00:00:06,NaN,00:00:00,TRUNK,...,00:00:05,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Принятый,Динамический коллтрекинг,2023-10-04 20:02:36,7.906796e+10,Нецелевой контакт,Media108 | Павелецкая Сити | direct | Яндекс.Д...,00:00:03,1.694681e+18,00:00:00,TRUNK,...,00:00:02,00:00:01,NaN,yandex,m108_pavcity_brand_net_mmo_exp_A|96311163,cpc,id|47205529851_47205529851|cid|96311163|gid|52...,купить квартиру павелецкая сити циан,NaN,NaN
2,Принятый,Аналитика,2023-10-04 19:49:06,7.916059e+10,NaN,MRGroup | Павелецкая Сити | seo | Google Поиск,00:20:46,1.696437e+18,00:00:00,TRUNK,...,00:20:46,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Потерянный,Автоперезвон по заявкам,2023-10-04 19:39:00,7.916059e+10,Обработано - Нецелевой контакт,MRGroup | Павелецкая Сити | seo | Google Поиск,00:00:33,1.696437e+18,00:00:33,Обратный звонок (SIP<1),...,00:00:00,00:00:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Потерянный,Автоперезвон по заявкам,2023-10-04 19:01:47,7.915439e+10,Не обработано - Нецелевой контакт,Media108 | Павелецкая Сити | direct | Яндекс.Д...,00:00:48,1.690479e+18,00:00:48,Обратный звонок (SIP<1),...,00:00:00,00:00:48,NaN,yandex,m108_pavcity_retarget_neotkaz_net_mmo_exp_A|96...,cpc,id|55165192_55165192|cid|96311169|gid|52884521...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1513,Потерянный,Звонки из VK Ads,2023-08-15 09:08:26,7.953435e+10,Не обработано - Нецелевой контакт,Media108 | Павелецкая Сити | social | VK Ads,00:00:55,NaN,00:00:55,Обратный звонок (SIP<1),...,00:00:00,00:00:55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1514,Принятый,Динамический коллтрекинг,2023-08-15 09:01:30,7.903290e+10,Нецелевой контакт,Media108 | Павелецкая Сити | direct | Яндекс.Д...,00:00:03,1.692030e+18,00:00:00,TRUNK,...,00:00:02,00:00:01,NaN,yandex,master|90303913,cpc,id|50275842_50275842|cid|90303913|gid|52363634...,NaN,NaN,NaN
1515,Принятый,Аналитика,2023-08-15 09:01:23,7.925407e+10,Нецелевой контакт,Media108 | Павелецкая Сити | social | VK Ads,00:00:07,NaN,00:00:00,TRUNK,...,00:00:07,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1516,Принятый,Звонки из VK Ads,2023-08-15 09:01:02,7.909630e+10,NaN,Media108 | Павелецкая Сити | social | VK Ads,00:00:38,NaN,00:00:32,Обратный звонок (SIP<1),...,00:00:06,00:00:32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_Paveletskaya2.Теги.unique()

array(['Нецелевой контакт', nan, 'Обработано - Нецелевой контакт',
       'Не обработано - Нецелевой контакт', 'Целевой_М108',
       'Нецелевой контакт - Целевой_М108',
       'Не обработано - Нецелевой контакт - Целевой_М108', 'тест'],
      dtype=object)

In [ ]:
df_Paveletskaya2 = df_Paveletskaya2.dropna(subset = ['Теги'])
df_Paveletskaya2 = df_Paveletskaya2.dropna(subset = ['Чистая длительность разговора'])

In [ ]:
df_Paveletskaya2.loc[~df_Paveletskaya2['Теги'].str.contains('Целевой_М108', case=False), 'Теги'] = 'Нецелевой'

In [ ]:
df_Paveletskaya2['Теги'] = df_Paveletskaya2['Теги'].astype(str)
df_Paveletskaya2['Теги'] = df_Paveletskaya2['Теги'].apply(lambda x: 'Целевой' if 'целевой_м108' in x.lower() else x)
df_Paveletskaya2['Теги'] = df_Paveletskaya2['Теги'].apply(lambda x: 'Нецелевой' if 'Нецелевой' in x else x)

In [ ]:
audio_folder = '/content/Записи Павелецкая сити'
audio_files = os.listdir(audio_folder)
session_ids = [re.findall(r'session_(\d+)', file)[0] for file in audio_files]
unique_ids = df_Paveletskaya2['Идентификатор сессии звонка'].unique()
unique_ids_in_files = [id for id in unique_ids if str(id) in session_ids]
df_Paveletskaya2 = df_Paveletskaya2[df_Paveletskaya2['Идентификатор сессии звонка'].isin(unique_ids_in_files)]

In [ ]:
df_Paveletskaya2['Чистая длительность разговора'] = pd.to_timedelta(df_Paveletskaya2['Чистая длительность разговора'])
df_Paveletskaya2 = df_Paveletskaya2[~((df_Paveletskaya2['Чистая длительность разговора'] < datetime.timedelta(seconds = 75)) & (df_Paveletskaya2['Теги'] == 'Целевой'))]

In [ ]:
targeted_rows = df_Paveletskaya2[df_Paveletskaya2['Теги'] == 'Целевой']
df_Targeted_Sample = targeted_rows.sample(frac=0.1, random_state=1)
nontargeted_rows = df_Paveletskaya2[df_Paveletskaya2['Теги'] == 'Нецелевой']
df_NonTargeted_Sample = nontargeted_rows.sample(frac=0.1, random_state=1)
df_Test_Pav2 = pd.concat([df_Targeted_Sample, df_NonTargeted_Sample])
df_Paveletskaya2 = df_Paveletskaya2.drop(df_Test_Pav2.index)
df_Test_Pav2.to_csv('df_Test_Pav2.csv', index=False)

In [ ]:
df_Paveletskaya2 = df_Paveletskaya2.reset_index(drop = True)

In [ ]:
df_Paveletskaya2

,Статус,Тип,Дата и время,Номер абонента,Теги,Рекламная кампания,Длительность звонка,Client ID Яндекс.Метрика,Длительность ожидания ответа,Сотрудник,...,Чистая длительность разговора,Полная длительность ожидания,Идентификатор звонка во внешней системе,Расширенная UTM-метка Source,Расширенная UTM-метка Campaign,Расширенная UTM-метка Medium,Расширенная UTM-метка Content,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid
0,Принятый,Аналитика,2023-10-04 22:46:01,7.968536e+10,Нецелевой,Media108 | Павелецкая Сити | catalog | Google ...,00:00:06,NaN,00:00:00,TRUNK,...,0 days 00:00:05,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Принятый,Динамический коллтрекинг,2023-10-04 20:02:36,7.906796e+10,Нецелевой,Media108 | Павелецкая Сити | direct | Яндекс.Д...,00:00:03,1.694681e+18,00:00:00,TRUNK,...,0 days 00:00:02,00:00:01,NaN,yandex,m108_pavcity_brand_net_mmo_exp_A|96311163,cpc,id|47205529851_47205529851|cid|96311163|gid|52...,купить квартиру павелецкая сити циан,NaN,NaN
2,Принятый,Аналитика,2023-10-04 17:09:53,7.925446e+10,Нецелевой,Media108 | Павелецкая Сити | cpa | IMetik,00:00:04,NaN,00:00:00,TRUNK,...,0 days 00:00:04,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Принятый,Динамический коллтрекинг,2023-10-04 16:27:50,7.906787e+10,Нецелевой,Media108 | Павелецкая Сити | direct | Яндекс.Д...,00:00:20,1.696424e+18,00:00:00,TRUNK,...,0 days 00:00:20,00:00:00,NaN,yandex,m108_pavcity_brand_net_mmo_exp_A|96311163,cpc,id|47205529877_47205529877|cid|96311163|gid|52...,павелецкая сити 2 очередь,NaN,NaN
4,Принятый,Аналитика,2023-10-04 10:45:40,7.924912e+10,Нецелевой,MRGroup | Павелецкая Сити | offline | Медиафас...,00:00:05,NaN,00:00:01,TRUNK,...,0 days 00:00:04,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,Принятый,Аналитика,2023-08-15 10:15:54,7.495297e+10,Нецелевой,Artics | Павелецкая Сити | offline | Планировка,00:00:26,NaN,00:00:01,TRUNK,...,0 days 00:00:25,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733,Принятый,Аналитика,2023-08-15 10:01:54,7.918339e+10,Нецелевой,Media108 | Павелецкая Сити | xml | M2,00:00:10,NaN,00:00:00,TRUNK,...,0 days 00:00:10,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
734,Принятый,Аналитика,2023-08-15 09:55:26,7.918339e+10,Нецелевой,MRGroup | Павелецкая Сити | seo | Google Поиск,00:00:10,NaN,00:00:01,TRUNK,...,0 days 00:00:09,00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
735,Принятый,Динамический коллтрекинг,2023-08-15 09:01:30,7.903290e+10,Нецелевой,Media108 | Павелецкая Сити | direct | Яндекс.Д...,00:00:03,1.692030e+18,00:00:00,TRUNK,...,0 days 00:00:02,00:00:01,NaN,yandex,master|90303913,cpc,id|50275842_50275842|cid|90303913|gid|52363634...,NaN,NaN,NaN


In [ ]:
df_Paveletskaya2.Теги.unique()

array(['Нецелевой', 'Целевой'], dtype=object)

In [ ]:
counts = df_Paveletskaya2['Теги'].value_counts()
print(counts)

Целевой      402
Нецелевой    335
Name: Теги, dtype: int64


In [ ]:
df_Paveletskaya2.to_csv('df_Paveletskaya_дополнительная.csv', index = False)

##Primavera.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import zipfile
import os
import re

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи звонков_primavera-20231026T154044Z-001.zip'

In [ ]:
df_Primavera = pd.read_excel('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты необработанные/Primavera.xlsx')
df_Primavera.to_csv('Primavera(Необработанная).csv', index = False)
path = '/content/Primavera(Необработанная).csv'
df_Primavera = pd.read_csv(path)

In [ ]:
df_Primavera

,Статус,Тип,Дата и время,Номер абонента,Рекламная кампания,Длительность звонка,Длительность ожидания ответа,Сотрудник,UTM-метка Campaign,Теги,...,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid,OS-метка service-name,OS-метка campaign-id,OS-метка ad-id,OS-метка source-id,Unnamed: 74,Unnamed: 75,Unnamed: 76
0,Принятый,Динамический коллтрекинг,2023-07-16 19:33:02,79254254955,Яндекс Директ,00:00:34,00:00:06,Operator5Metrium,m108|poligon_village_mo_rsy_mmo|88323340|context,Нецелевой контакт_М108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Потерянный,Омниканальный виджет,2023-07-16 19:26:06,79878241043,Яндекс Директ,00:00:12,00:00:12,NaN,m108|interes_realty_finans_rsy_rf|82276340|con...,Не обработано - Нецелевой контакт - Повтор_М108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Потерянный,Омниканальный виджет,2023-07-16 19:10:50,79878241043,Яндекс Директ,00:00:14,00:00:14,NaN,m108|interes_realty_finans_rsy_rf|82276340|con...,Не обработано - Нецелевой контакт - Повтор_М108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Потерянный,Омниканальный виджет,2023-07-16 19:05:35,79878241043,Яндекс Директ,00:00:12,00:00:12,NaN,m108|interes_realty_finans_rsy_rf|82276340|con...,Не обработано - Нецелевой контакт - Нецелевой ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Потерянный,Динамический коллтрекинг,2023-07-16 19:03:23,79031161766,Яндекс Директ,00:00:04,00:00:04,NaN,m108|competitors_onega_rsy_mmo|90827470|context,Не обработано - Нецелевой контакт - Нецелевой ...,...,жк акватория москва купить квартиру от застрой...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2887,Принятый,Динамический коллтрекинг,2023-05-01 12:48:35,79372616437,Яндекс Директ,00:00:48,00:00:20,Manager3OneMoscow,m108|interes_realty_finans_rsy_mmo|82276330|co...,Нецелевой контакт_М108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2888,Принятый,Динамический коллтрекинг,2023-05-01 11:00:13,79014476062,Ястреб,00:00:15,00:00:05,Operator3Metrium,m108|april2023,Нецелевой контакт - Нецелевой контакт_М108,...,yan:tgb:m100021m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2889,Принятый,Аналитика,2023-05-01 10:25:44,79605015550,kvartirny-control.ru | Медийная реклама,00:09:31,00:00:05,Operator5VSN,NaN,Целевой_М108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2890,Принятый,Динамический коллтрекинг,2023-05-01 10:11:26,79260655992,Яндекс Директ,00:01:19,00:00:16,Manager3OneMoscow,m108|brand_search_mmo|75730306|search,Повтор_М108,...,примавера,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_Primavera.Теги.unique()

array(['Нецелевой контакт_М108',
       'Не обработано - Нецелевой контакт - Повтор_М108',
       'Не обработано - Нецелевой контакт - Нецелевой контакт_М108',
       'Целевой_М108', 'Целевой_М108 - Встреча',
       'Нецелевой контакт - Нецелевой контакт_М108',
       'Нецелевой контакт - Повтор_М108',
       'Не обработано - Целевой_М108 - Нецелевой контакт', 'Повтор_М108',
       'Обработано - Нецелевой контакт - Повтор_М108',
       'Целевой_М108 - ПБ - Встреча - ДДУ',
       'Обработано - Нецелевой контакт - Нецелевой контакт_М108',
       'Целевой_М108 - Обработано - Нецелевой контакт',
       'Спам - Нецелевой контакт', 'Целевой_М108 - Нецелевой контакт',
       'Целевой_М108 - Обработано - Встреча - Нецелевой контакт',
       'Целевой_М108 - Повтор_М108 - по критериям площадки',
       'Целевой_М108 - по критериям площадки', 'Целевой_М108 - ПБ',
       'Не обработано - Целевой_М108 - ПБ - ДДУ - Нецелевой контакт',
       'Не обработано - Целевой_М108 - Встреча - Нецелевой контак

In [ ]:
df_Primavera = df_Primavera.dropna(subset = ['Теги'])
df_Primavera = df_Primavera.dropna(subset = ['Чистая длительность разговора'])

In [ ]:
df_Primavera.loc[~df_Primavera['Теги'].str.contains('Целевой_М108', case=False), 'Теги'] = 'Нецелевой'

In [ ]:
df_Primavera['Теги'] = df_Primavera['Теги'].astype(str)
df_Primavera['Теги'] = df_Primavera['Теги'].apply(lambda x: 'Целевой' if 'целевой_м108' in x.lower() else x)
df_Primavera['Теги'] = df_Primavera['Теги'].apply(lambda x: 'Нецелевой' if 'Нецелевой' in x else x)

In [ ]:
audio_folder = '/content/Записи звонков_primavera'
audio_files = os.listdir(audio_folder)
session_ids = [re.findall(r'session_(\d+)', file)[0] for file in audio_files]
unique_ids = df_Primavera['Идентификатор сессии звонка'].unique()
unique_ids_in_files = [id for id in unique_ids if str(id) in session_ids]
df_Primavera = df_Primavera[df_Primavera['Идентификатор сессии звонка'].isin(unique_ids_in_files)]

In [ ]:
df_Primavera = df_Primavera.copy()
df_Primavera['Чистая длительность разговора'] = pd.to_timedelta(df_Primavera['Чистая длительность разговора'])
df_Primavera = df_Primavera[~((df_Primavera['Чистая длительность разговора'] < datetime.timedelta(seconds = 75)) & (df_Primavera['Теги'] == 'Целевой'))]

In [ ]:
targeted_rows = df_Primavera[df_Primavera['Теги'] == 'Целевой']
df_Targeted_Sample = targeted_rows.sample(frac=0.1, random_state=1)
nontargeted_rows = df_Primavera[df_Primavera['Теги'] == 'Нецелевой']
df_NonTargeted_Sample = nontargeted_rows.sample(frac=0.1, random_state=1)
df_Test_Prim1 = pd.concat([df_Targeted_Sample, df_NonTargeted_Sample])
df_Primavera = df_Primavera.drop(df_Test_Prim1.index)
df_Test_Prim1.to_csv('df_Test_Prim1.csv', index=False)

In [ ]:
df_Primavera = df_Primavera.reset_index(drop = True)

In [ ]:
df_Primavera

,Статус,Тип,Дата и время,Номер абонента,Рекламная кампания,Длительность звонка,Длительность ожидания ответа,Сотрудник,UTM-метка Campaign,Теги,...,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid,OS-метка service-name,OS-метка campaign-id,OS-метка ad-id,OS-метка source-id,Unnamed: 74,Unnamed: 75,Unnamed: 76
0,Принятый,Динамический коллтрекинг,2023-07-16 19:33:02,79254254955,Яндекс Директ,00:00:34,00:00:06,Operator5Metrium,m108|poligon_village_mo_rsy_mmo|88323340|context,Нецелевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Принятый,Аналитика,2023-07-16 18:57:30,79066744592,Авито | Базы | Звонок,00:00:33,00:00:30,Aleksandra Zhigareva,NaN,Нецелевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Принятый,Аналитика,2023-07-16 17:24:06,79053937606,Авито | Базы | Звонок,00:00:53,00:00:27,Tatyana Denisova,NaN,Нецелевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Принятый,Аналитика,2023-07-16 16:30:52,79857650508,Яндекс Визитка,00:02:36,00:00:06,Yana Solovyova,NaN,Целевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Принятый,Аналитика,2023-07-16 14:58:59,79851629113,Яндекс.Карты (расширенная карточка),00:02:15,00:00:03,Operator3Metrium,NaN,Целевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,Принятый,Аналитика,2023-05-01 14:40:29,74952969962,Циан | Базы | Звонок,00:00:12,00:00:05,Operator5VSN,NaN,Нецелевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1300,Принятый,Динамический коллтрекинг,2023-05-01 12:48:35,79372616437,Яндекс Директ,00:00:48,00:00:20,Manager3OneMoscow,m108|interes_realty_finans_rsy_mmo|82276330|co...,Нецелевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1301,Принятый,Динамический коллтрекинг,2023-05-01 11:00:13,79014476062,Ястреб,00:00:15,00:00:05,Operator3Metrium,m108|april2023,Нецелевой,...,yan:tgb:m100021m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1302,Принятый,Аналитика,2023-05-01 10:25:44,79605015550,kvartirny-control.ru | Медийная реклама,00:09:31,00:00:05,Operator5VSN,NaN,Целевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_Primavera.Теги.unique()

array(['Нецелевой', 'Целевой'], dtype=object)

In [ ]:
counts = df_Primavera['Теги'].value_counts()
print(counts)

Нецелевой    769
Целевой      535
Name: Теги, dtype: int64


In [ ]:
df_Primavera.to_csv('df_Primavera.csv', index=False)

##Дополнительный DF по Primavera.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import zipfile
import os
import re

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи_Primavera_дополнительные-20231028T005838Z-001.zip'

In [ ]:
df_Primavera2 = pd.read_excel('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты необработанные/Примавера_дополнительные.xlsx')
df_Primavera2.to_csv('Primavera(Необработанная2).csv', index = False)
path = '/content/Primavera(Необработанная2).csv'
df_Primavera2 = pd.read_csv(path)

In [ ]:
df_Primavera2

,Статус,Тип,Дата и время,Номер абонента,Рекламная кампания,Длительность звонка,Номер обращения,Теги,Сотрудник,Первая рекламная кампания,...,Расширенная UTM-метка Campaign,Расширенная UTM-метка Medium,Расширенная UTM-метка Content,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid,OS-метка service-name,OS-метка campaign-id,OS-метка ad-id,OS-метка source-id
0,Принятый,Аналитика,2023-10-04 19:47:03,7.962401e+10,Roomberry | Лидогенерация,00:05:24,1,NaN,Operator7Metrium,Посетители без рекламной кампании,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Принятый,Омниканальный виджет,2023-10-04 19:40:04,7.938796e+10,Яндекс Директ,00:00:34,4346,NaN,Svetlana Aristova,Яндекс Директ,...,m108|competitors_new_rsy_mmo|92541971|context,cpc,5255717580|14773407867|46335276624|hoz-house.r...,river park кутузовский цена,NaN,NaN,NaN,NaN,NaN,NaN
2,Принятый,Аналитика,2023-10-04 18:13:43,7.926400e+10,Яндекс.Карты (расширенная карточка),00:01:57,15,NaN,Operator5Metrium,{Google SEO},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Потерянный,Омниканальный виджет,2023-10-04 18:11:55,7.946456e+10,Посетители без рекламной кампании,00:00:03,6,Не обработано - Нецелевой контакт,NaN,Посетители без рекламной кампании,...,m108,cpc,NaN,yan:video:m100022m,NaN,NaN,NaN,NaN,NaN,NaN
4,Потерянный,Омниканальный виджет,2023-10-04 17:56:51,7.946456e+10,Посетители без рекламной кампании,00:00:03,5,Не обработано - Нецелевой контакт,NaN,Посетители без рекламной кампании,...,m108,cpc,NaN,yan:video:m100022m,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,Принятый,Аналитика,2023-09-01 12:05:19,7.919994e+10,Реферальный трафик,00:00:22,1,Нецелевой контакт - Нецелевой контакт_М108,Operator7Metrium,Посетители без рекламной кампании,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1519,Принятый,Аналитика,2023-09-01 11:41:53,7.937800e+10,Яндекс.Карты (расширенная карточка),00:02:02,1,Целевой_М108,Ekaterina Rumyantseva,Посетители без рекламной кампании,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1520,Принятый,Звонок ВАТС,2023-09-01 11:16:24,7.916116e+10,Reffection | Лидогенерация,00:05:19,2,Целевой_М108,Operator7Metrium,Посетители без рекламной кампании,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1521,Принятый,Аналитика,2023-09-01 09:25:53,7.926468e+10,Циан | Базы | Звонок,00:07:07,1,Целевой_М108,Operator3Metrium,Посетители без рекламной кампании,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_Primavera2.Теги.unique()

array([nan, 'Не обработано - Нецелевой контакт',
       'Обработано - Нецелевой контакт',
       'Нецелевой контакт - тест - Нецелевой контакт_М108',
       'Тест - Нецелевой контакт - Нецелевой контакт_М108',
       'Нецелевой контакт_М108', 'Запросить статус_М108', 'Повтор_М108',
       'Обработано - Запросить статус_М108 - Нецелевой контакт',
       'Не обработано - Нецелевой контакт - Повтор_М108',
       'Не обработано - Запросить статус_М108 - Нецелевой контакт',
       'Нецелевой контакт - Нецелевой контакт_М108', 'Целевой_М108',
       'Нецелевой контакт - Повтор_М108',
       'Запросить статус_М108 - Нецелевой контакт',
       'Обработано - Нецелевой контакт - Повтор_М108',
       'Обработано - Нецелевой контакт - Нецелевой контакт_М108',
       'Не обработано - Нецелевой контакт - Нецелевой контакт_М108',
       'Целевой_М108 - Встреча', 'Целевой_М108 - Нецелевой контакт',
       'Спам - Нецелевой контакт - Нецелевой контакт_М108',
       'Целевой_М108 - Обработано - Нецелево

In [ ]:
df_Primavera2 = df_Primavera2.dropna(subset = ['Теги'])
df_Primavera2 = df_Primavera2.dropna(subset = ['Чистая длительность разговора'])

In [ ]:
df_Primavera2.loc[~df_Primavera2['Теги'].str.contains('Целевой_М108', case=False), 'Теги'] = 'Нецелевой'

In [ ]:
df_Primavera2['Теги'] = df_Primavera2['Теги'].astype(str)
df_Primavera2['Теги'] = df_Primavera2['Теги'].apply(lambda x: 'Целевой' if 'целевой_м108' in x.lower() else x)
df_Primavera2['Теги'] = df_Primavera2['Теги'].apply(lambda x: 'Нецелевой' if 'Нецелевой' in x else x)

In [ ]:
audio_folder = '/content/Записи Primavera'
audio_files = os.listdir(audio_folder)
session_ids = [re.findall(r'session_(\d+)', file)[0] for file in audio_files]
unique_ids = df_Primavera2['Идентификатор сессии звонка'].unique()
unique_ids_in_files = [id for id in unique_ids if str(id) in session_ids]
df_Primavera2 = df_Primavera2[df_Primavera2['Идентификатор сессии звонка'].isin(unique_ids_in_files)]

In [ ]:
df_Primavera2 = df_Primavera2.copy()
df_Primavera2['Чистая длительность разговора'] = pd.to_timedelta(df_Primavera2['Чистая длительность разговора'])
df_Primavera2 = df_Primavera2[~((df_Primavera2['Чистая длительность разговора'] < datetime.timedelta(seconds = 75)) & (df_Primavera2['Теги'] == 'Целевой'))]

In [ ]:
targeted_rows = df_Primavera2[df_Primavera2['Теги'] == 'Целевой']
df_Targeted_Sample = targeted_rows.sample(frac=0.1, random_state=1)
nontargeted_rows = df_Primavera2[df_Primavera2['Теги'] == 'Нецелевой']
df_NonTargeted_Sample = nontargeted_rows.sample(frac=0.1, random_state=1)
df_Test_Prim2 = pd.concat([df_Targeted_Sample, df_NonTargeted_Sample])
df_Primavera2 = df_Primavera2.drop(df_Test_Prim2.index)
df_Test_Prim2.to_csv('df_Test_Prim2.csv', index=False)

In [ ]:
df_Primavera2 = df_Primavera2.reset_index(drop = True)

In [ ]:
df_Primavera2

,Статус,Тип,Дата и время,Номер абонента,Рекламная кампания,Длительность звонка,Номер обращения,Теги,Сотрудник,Первая рекламная кампания,...,Расширенная UTM-метка Campaign,Расширенная UTM-метка Medium,Расширенная UTM-метка Content,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid,OS-метка service-name,OS-метка campaign-id,OS-метка ad-id,OS-метка source-id
0,Принятый,Звонок ВАТС,2023-10-04 16:11:12,7.968604e+10,Интеграция: Ивлеева_telegram,00:00:16,636,Нецелевой,Ekaterina Rumyantseva,Google SEO,...,m108,september23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Принятый,Динамический коллтрекинг,2023-10-04 16:05:52,7.968604e+10,Интеграция: Ивлеева_telegram,00:00:21,635,Нецелевой,Aleksandra Khrustaleva,Google SEO,...,m108,september23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Потерянный,Динамический коллтрекинг,2023-10-04 15:58:55,7.905584e+10,Google SEO,00:00:04,29,Нецелевой,Operator5Metrium,Google SEO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Принятый,Аналитика,2023-10-04 15:51:42,7.920381e+10,Novostroy-m | Звонок,00:04:41,1,Нецелевой,Ekaterina Rumyantseva,Посетители без рекламной кампании,...,NaN,week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Принятый,Омниканальный виджет,2023-10-04 15:28:02,7.918668e+10,RTB_M108,00:00:36,7,Нецелевой,Tatyana Denisova,RTB_M108,...,m108|nov2022,cpm,premium_class,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,Принятый,Аналитика,2023-09-01 15:28:04,7.916716e+10,Прямой заход,00:28:57,4,Нецелевой,Operator7Metrium,Посетители без рекламной кампании,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
636,Принятый,Аналитика,2023-09-01 12:35:45,7.977681e+10,Google SEO,00:01:16,1,Нецелевой,Yana Solovyova,Посетители без рекламной кампании,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
637,Принятый,Аналитика,2023-09-01 12:05:19,7.919994e+10,Реферальный трафик,00:00:22,1,Нецелевой,Operator7Metrium,Посетители без рекламной кампании,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638,Принятый,Аналитика,2023-09-01 11:41:53,7.937800e+10,Яндекс.Карты (расширенная карточка),00:02:02,1,Целевой,Ekaterina Rumyantseva,Посетители без рекламной кампании,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_Primavera2.Теги.unique()

array(['Нецелевой', 'Целевой'], dtype=object)

In [ ]:
counts = df_Primavera2['Теги'].value_counts()
print(counts)

Нецелевой    398
Целевой      242
Name: Теги, dtype: int64


In [ ]:
df_Primavera2.to_csv('df_Primavera_дополнительная.csv', index=False)

##Headliner.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import zipfile
import os
import re

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи звонков_хедлайнер-20231026T173308Z-001.zip'

In [ ]:
df_Headliner = pd.read_excel('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты необработанные/Headliner.xlsx')
df_Headliner.to_csv('Headliner(Необработанная).csv', index = False)
path = '/content/Headliner(Необработанная).csv'
df_Headliner = pd.read_csv(path)

In [ ]:
df_Headliner

,Статус,Тип,Дата и время,Номер абонента,Виртуальный номер,Рекламная кампания,Длительность звонка,Длительность ожидания ответа,Причина завершения,Сценарий,...,Внутренний звонок,ФИО контакта,Имя тренера,"Группа, выбранная посетителем",Регион номера абонента,Страна номера абонента,Контакт из CRM,Unnamed: 72,Unnamed: 73,Unnamed: 74
0,Принятый,Динамический коллтрекинг,2023-07-19 17:12:54,7.999135e+10,74951322907,Поисковое продвижение,00:06:39,00:00:05,Абонент разорвал соединение,.Последовательный обзвон,...,Нет,NaN,NaN,NaN,Республика Башкортостан,РФ,NaN,NaN,NaN,NaN
1,Принятый,Динамический коллтрекинг,2023-07-19 17:10:22,7.999135e+10,74951322907,Поисковое продвижение,00:01:36,00:00:46,Абонент разорвал соединение,.Последовательный обзвон,...,Нет,NaN,NaN,NaN,Республика Башкортостан,РФ,NaN,NaN,NaN,NaN
2,Принятый,Звонок ВАТС,2023-07-19 17:01:50,7.983509e+10,74950320897,Медийная реклама // msk.novostroy-gid.ru // Зв...,00:01:59,00:00:05,Абонент разорвал соединение,.Последовательный обзвон,...,Нет,NaN,NaN,NaN,Красноярский край,РФ,NaN,NaN,NaN,NaN
3,Принятый,Звонок ВАТС,2023-07-19 16:52:09,7.915240e+10,74950854280,ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,00:07:33,00:00:05,Абонент разорвал соединение,74959339929 ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,...,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN,NaN,NaN,NaN
4,Принятый,Звонок ВАТС,2023-07-19 16:50:56,7.916102e+10,74950320897,Медийная реклама // msk.novostroy-gid.ru // Зв...,00:00:52,00:00:05,Сотрудник разорвал соединение,.Последовательный обзвон,...,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3853,Принятый,Динамический коллтрекинг,2023-05-08 12:51:08,7.499704e+10,74954313905,Яндекс Дисплей,00:00:36,00:00:31,Абонент разорвал соединение,.Последовательный обзвон,...,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN,NaN,NaN,NaN
3854,Принятый,Звонок ВАТС,2023-05-08 12:35:55,7.962280e+10,74953244320,Медийная реклама // Roomberry // звонок с карт...,00:01:47,00:00:01,Абонент разорвал соединение,Кортрос,...,Нет,NaN,NaN,NaN,Камчатский край,РФ,NaN,NaN,NaN,NaN
3855,Принятый,Звонок ВАТС,2023-05-08 12:34:46,7.962280e+10,74953244320,Медийная реклама // Roomberry // звонок с карт...,00:00:09,00:00:01,Абонент разорвал соединение,Кортрос,...,Нет,NaN,NaN,NaN,Камчатский край,РФ,NaN,NaN,NaN,NaN
3856,Принятый,Звонок ВАТС,2023-05-08 11:43:52,7.982769e+10,74951060379,Медийная реклама // kvartirny-control // Спец,00:06:41,00:00:01,Сотрудник разорвал соединение,.Последовательный обзвон,...,Нет,NaN,NaN,NaN,Свердловская обл.,РФ,NaN,NaN,NaN,NaN


In [ ]:
df_Headliner.Теги.unique()

array(['Первичный целевой - 3 очередь - 1-ком',
       'спорный - нет соединения',
       'спорный - номер взят не дозвонились  до ОП',
       'постпродажное обслуживание - Условно целевой - Б.хаус',
       'спорный - клиент просил перезвонить',
       'постпродажное обслуживание - i love - Условно целевой',
       'повторная покупка - Вторичный целевой - 2 очередь - 1-ком',
       'Первичный целевой - 2 очередь - 3 очередь - 2-ком',
       'Лид - не проходит по бюджету - 3 очередь - Условно целевой - 1-ком',
       '3 очередь - 2-ком - Условно целевой',
       'спорный - связь прервалась',
       'Первичный целевой - студия - 3 очередь',
       'Нецелевой звонок - сотрудничество', 'тест',
       'Вторичный целевой - 2 очередь - 2-ком - балкон - 3-ком',
       'Первичный целевой - ипотека - студия - 3 очередь',
       'повторная покупка - Вторичный целевой - 2 очередь - 3 очередь - 1-ком',
       'Нецелевой звонок - ошибка', 'i love - 2-ком - Условно целевой',
       'интересуют скидки

In [ ]:
df_Headliner = df_Headliner.dropna(subset = ['Теги'])
df_Headliner = df_Headliner.dropna(subset = ['Чистая длительность разговора'])

In [ ]:
pattern = r'Первичный целевой|Вторичный целевой'
df_Headliner.loc[~df_Headliner['Теги'].str.contains(pattern, case=False, regex=True), 'Теги'] = 'Нецелевой'

In [ ]:
df_Headliner['Теги'] = df_Headliner['Теги'].astype(str)
df_Headliner['Теги'] = df_Headliner['Теги'].apply(lambda x: 'Целевой' if 'первичный целевой' in x.lower() or 'вторичный целевой' in x.lower() else x)
df_Headliner['Теги'] = df_Headliner['Теги'].apply(lambda x: 'Нецелевой' if 'Нецелевой' in x else x)

In [ ]:
audio_folder = '/content/Записи звонков_хедлайнер'
audio_files = os.listdir(audio_folder)
session_ids = [re.findall(r'session_(\d+)', file)[0] for file in audio_files]
unique_ids = df_Headliner['Идентификатор сессии звонка'].unique()
unique_ids_in_files = [id for id in unique_ids if str(id) in session_ids]
df_Headliner = df_Headliner[df_Headliner['Идентификатор сессии звонка'].isin(unique_ids_in_files)]

In [ ]:
df_Headliner['Чистая длительность разговора'] = pd.to_timedelta(df_Headliner['Чистая длительность разговора'])
df_Headliner = df_Headliner[~((df_Headliner['Чистая длительность разговора'] < datetime.timedelta(seconds = 75)) & (df_Headliner['Теги'] == 'Целевой'))]

In [ ]:
targeted_rows = df_Headliner[df_Headliner['Теги'] == 'Целевой']
df_Targeted_Sample = targeted_rows.sample(frac=0.1, random_state=1)
nontargeted_rows = df_Headliner[df_Headliner['Теги'] == 'Нецелевой']
df_NonTargeted_Sample = nontargeted_rows.sample(frac=0.1, random_state=1)
df_Test_Head1 = pd.concat([df_Targeted_Sample, df_NonTargeted_Sample])
df_Headliner = df_Headliner.drop(df_Test_Head1.index)
df_Test_Head1.to_csv('df_Test_Head1.csv', index=False)

In [ ]:
df_Headliner = df_Headliner.reset_index(drop = True)

In [ ]:
df_Headliner

,Статус,Тип,Дата и время,Номер абонента,Виртуальный номер,Рекламная кампания,Длительность звонка,Длительность ожидания ответа,Причина завершения,Сценарий,...,Внутренний звонок,ФИО контакта,Имя тренера,"Группа, выбранная посетителем",Регион номера абонента,Страна номера абонента,Контакт из CRM,Unnamed: 72,Unnamed: 73,Unnamed: 74
0,Принятый,Динамический коллтрекинг,2023-07-19 17:10:22,7.999135e+10,74951322907,Поисковое продвижение,00:01:36,00:00:46,Абонент разорвал соединение,.Последовательный обзвон,...,Нет,NaN,NaN,NaN,Республика Башкортостан,РФ,NaN,NaN,NaN,NaN
1,Принятый,Звонок ВАТС,2023-07-19 17:01:50,7.983509e+10,74950320897,Медийная реклама // msk.novostroy-gid.ru // Зв...,00:01:59,00:00:05,Абонент разорвал соединение,.Последовательный обзвон,...,Нет,NaN,NaN,NaN,Красноярский край,РФ,NaN,NaN,NaN,NaN
2,Принятый,Звонок ВАТС,2023-07-19 16:52:09,7.915240e+10,74950854280,ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,00:07:33,00:00:05,Абонент разорвал соединение,74959339929 ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,...,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN,NaN,NaN,NaN
3,Принятый,Звонок ВАТС,2023-07-19 16:48:24,7.916689e+10,74950854280,ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,00:00:58,00:00:05,Сотрудник разорвал соединение,74959339929 ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,...,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN,NaN,NaN,NaN
4,Принятый,Звонок ВАТС,2023-07-19 16:41:49,7.915319e+10,74950854280,ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,00:01:07,00:00:05,Сотрудник разорвал соединение,74959339929 ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,...,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765,Принятый,Динамический коллтрекинг,2023-05-08 12:51:08,7.499704e+10,74954313905,Яндекс Дисплей,00:00:36,00:00:31,Абонент разорвал соединение,.Последовательный обзвон,...,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN,NaN,NaN,NaN
1766,Принятый,Звонок ВАТС,2023-05-08 12:35:55,7.962280e+10,74953244320,Медийная реклама // Roomberry // звонок с карт...,00:01:47,00:00:01,Абонент разорвал соединение,Кортрос,...,Нет,NaN,NaN,NaN,Камчатский край,РФ,NaN,NaN,NaN,NaN
1767,Принятый,Звонок ВАТС,2023-05-08 12:34:46,7.962280e+10,74953244320,Медийная реклама // Roomberry // звонок с карт...,00:00:09,00:00:01,Абонент разорвал соединение,Кортрос,...,Нет,NaN,NaN,NaN,Камчатский край,РФ,NaN,NaN,NaN,NaN
1768,Принятый,Звонок ВАТС,2023-05-08 11:43:52,7.982769e+10,74951060379,Медийная реклама // kvartirny-control // Спец,00:06:41,00:00:01,Сотрудник разорвал соединение,.Последовательный обзвон,...,Нет,NaN,NaN,NaN,Свердловская обл.,РФ,NaN,NaN,NaN,NaN


In [ ]:
df_Headliner.Теги.unique()

array(['Нецелевой', 'Целевой'], dtype=object)

In [ ]:
counts = df_Headliner['Теги'].value_counts()
print(counts)

Нецелевой    1213
Целевой       557
Name: Теги, dtype: int64


In [ ]:
df_Headliner.to_csv('df_Headliner.csv', index = False)

##Дополнительный DF по Headliner.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import zipfile
import os
import re

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи_Headliner_дополнительные-20231028T005642Z-001.zip'

In [ ]:
df_Headliner2 = pd.read_excel('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты необработанные/Headliner_дополнительные.xlsx')
df_Headliner2.to_csv('Headliner(Необработанная2).csv', index = False)
path = '/content/Headliner(Необработанная2).csv'
df_Headliner2 = pd.read_csv(path)

In [ ]:
df_Headliner2

,Статус,Тип,Дата и время,Номер абонента,Виртуальный номер,Рекламная кампания,Длительность звонка,Длительность ожидания ответа,Причина завершения,Сценарий,...,Домен реферера,Сайт,Оценка сотрудника,Внутренний звонок,ФИО контакта,Имя тренера,"Группа, выбранная посетителем",Регион номера абонента,Страна номера абонента,Контакт из CRM
0,Принятый,Звонок ВАТС,2023-10-05 10:26:28,79235257654,74950854280,ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,00:02:12,00:00:01,Сотрудник разорвал соединение,74959339929 ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,...,NaN,head-liner.ru,NaN,Нет,NaN,NaN,NaN,Кемеровская обл.,РФ,NaN
1,Принятый,Динамический коллтрекинг,2023-10-05 10:24:42,79235257654,74953249432,Поисковое продвижение,00:01:24,00:00:09,Абонент разорвал соединение,.Последовательный обзвон,...,www.google.com,head-liner.ru,NaN,Нет,NaN,NaN,NaN,Кемеровская обл.,РФ,NaN
2,Принятый,Динамический коллтрекинг,2023-10-05 10:23:07,79235257654,74953249432,Поисковое продвижение,00:00:38,00:00:09,Абонент разорвал соединение,.Последовательный обзвон,...,www.google.com,head-liner.ru,NaN,Нет,NaN,NaN,NaN,Кемеровская обл.,РФ,NaN
3,Потерянный,Динамический коллтрекинг,2023-10-05 10:22:18,79235257654,74953249432,Поисковое продвижение,00:00:06,00:00:06,Абонент разорвал соединение,.Последовательный обзвон,...,www.google.com,head-liner.ru,NaN,Нет,NaN,NaN,NaN,Кемеровская обл.,РФ,NaN
4,Принятый,Динамический коллтрекинг,2023-10-05 10:20:50,79067910613,74951629501,Яндекс Директ,00:01:58,00:00:08,Сотрудник разорвал соединение,.Последовательный обзвон,...,yandex.ru,head-liner.ru,NaN,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,Принятый,Звонок ВАТС,2023-08-23 10:20:52,79166964481,74950854280,ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,00:00:55,00:00:10,Абонент разорвал соединение,74959339929 ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,...,NaN,head-liner.ru,NaN,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN
1537,Принятый,Звонок ВАТС,2023-08-23 10:15:04,79166964481,74950854280,ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,00:01:00,00:00:09,Абонент разорвал соединение,74959339929 ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,...,NaN,head-liner.ru,NaN,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN
1538,Принятый,Звонок ВАТС,2023-08-23 10:08:09,79166964481,74950854280,ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,00:00:45,00:00:09,Абонент разорвал соединение,74959339929 ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,...,NaN,head-liner.ru,NaN,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN
1539,Принятый,Звонок ВАТС,2023-08-23 09:32:28,79161079278,74951063507,Базы // Яндекс Недвижимость,00:03:20,00:00:01,Абонент разорвал соединение,Кортрос,...,NaN,head-liner.ru,NaN,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN


In [ ]:
df_Headliner2.Теги.unique()

array([nan, 'Нецелевой контакт', 'спорный - нет соединения',
       'спорный - номер взят не дозвонились  до ОП',
       'Нецелевой звонок - ошибка',
       'Лид - спорный - номер взят не дозвонились  до ОП - 4-ком',
       'постпродажное обслуживание - Вторичный целевой',
       'Лид - номер взят не дозвонились  до ОП - 3-ком',
       'спорный - 2-ком - связь прервалась на ОП', 'Спам',
       'спорный - Коммерческая недвижимость - номер взят не дозвонились  до ОП',
       'спорный - связь прервалась',
       'Первичный целевой - 2 очередь - от застройщика - 1-ком',
       'Нецелевой звонок - сотрудничество',
       'Нецелевой звонок - не интересует объект',
       'спорный - менеджер сам перезвонит',
       'спорный - интересует график работы офиса',
       'спорный - номер взят не дозвонились  до ОП - заказ такси',
       'Лид - спорный - связь прервалась',
       'Первичный целевой - ипотека - интерес к объекту - не определился',
       'Первичный целевой - запись на просмотр - 3 оч

In [ ]:
df_Headliner2 = df_Headliner2.dropna(subset = ['Теги'])
df_Headliner2 = df_Headliner2.dropna(subset = ['Чистая длительность разговора'])

In [ ]:
pattern = r'Первичный целевой|Вторичный целевой'
df_Headliner2.loc[~df_Headliner2['Теги'].str.contains(pattern, case=False, regex=True), 'Теги'] = 'Нецелевой'

In [ ]:
df_Headliner2['Теги'] = df_Headliner2['Теги'].astype(str)
df_Headliner2['Теги'] = df_Headliner2['Теги'].apply(lambda x: 'Целевой' if 'первичный целевой' in x.lower() or 'вторичный целевой' in x.lower() else x)
df_Headliner2['Теги'] = df_Headliner2['Теги'].apply(lambda x: 'Нецелевой' if 'Нецелевой' in x else x)

In [ ]:
audio_folder = '/content/Записи Headliner'
audio_files = os.listdir(audio_folder)
session_ids = [re.findall(r'session_(\d+)', file)[0] for file in audio_files]
unique_ids = df_Headliner2['Идентификатор сессии звонка'].unique()
unique_ids_in_files = [id for id in unique_ids if str(id) in session_ids]
df_Headliner2 = df_Headliner2[df_Headliner2['Идентификатор сессии звонка'].isin(unique_ids_in_files)]

In [ ]:
df_Headliner2['Чистая длительность разговора'] = pd.to_timedelta(df_Headliner2['Чистая длительность разговора'])
df_Headliner2 = df_Headliner2[~((df_Headliner2['Чистая длительность разговора'] < datetime.timedelta(seconds = 75)) & (df_Headliner2['Теги'] == 'Целевой'))]

In [ ]:
targeted_rows = df_Headliner2[df_Headliner2['Теги'] == 'Целевой']
df_Targeted_Sample = targeted_rows.sample(frac=0.1, random_state=1)
nontargeted_rows = df_Headliner2[df_Headliner2['Теги'] == 'Нецелевой']
df_NonTargeted_Sample = nontargeted_rows.sample(frac=0.1, random_state=1)
df_Test_Head2 = pd.concat([df_Targeted_Sample, df_NonTargeted_Sample])
df_Headliner2 = df_Headliner2.drop(df_Test_Head2.index)
df_Test_Head2.to_csv('df_Test_Head2.csv', index=False)

In [ ]:
df_Headliner2 = df_Headliner2.reset_index(drop = True)

In [ ]:
df_Headliner2

,Статус,Тип,Дата и время,Номер абонента,Виртуальный номер,Рекламная кампания,Длительность звонка,Длительность ожидания ответа,Причина завершения,Сценарий,...,Домен реферера,Сайт,Оценка сотрудника,Внутренний звонок,ФИО контакта,Имя тренера,"Группа, выбранная посетителем",Регион номера абонента,Страна номера абонента,Контакт из CRM
0,Принятый,Динамический коллтрекинг,2023-10-04 20:45:39,74953846982,74951250252,Поисковое продвижение,00:01:13,00:00:08,Сотрудник разорвал соединение,.Последовательный обзвон,...,yandex.ru,head-liner.ru,NaN,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN
1,Принятый,Динамический коллтрекинг,2023-10-04 20:17:47,74953846982,74951250252,Поисковое продвижение,00:00:09,00:00:00,Абонент разорвал соединение,.Последовательный обзвон,...,yandex.ru,head-liner.ru,NaN,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN
2,Принятый,Звонок ВАТС,2023-10-04 18:31:13,79039764719,74951063507,Базы // Яндекс Недвижимость,00:01:39,00:00:01,Сотрудник разорвал соединение,Кортрос,...,NaN,head-liner.ru,NaN,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN
3,Принятый,Звонок ВАТС,2023-10-04 17:26:44,79166761425,74951810000,Звонки_оффлайн,00:00:49,00:00:01,Сотрудник разорвал соединение,.Последовательный обзвон,...,NaN,head-liner.ru,NaN,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN
4,Принятый,Звонок ВАТС,2023-10-04 17:10:21,79254460225,74950238364,Лидогенерация // Айметик,00:01:27,00:00:01,Сотрудник разорвал соединение,.Последовательный обзвон,...,NaN,head-liner.ru,NaN,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052,Принятый,Звонок ВАТС,2023-08-23 10:20:52,79166964481,74950854280,ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,00:00:55,00:00:10,Абонент разорвал соединение,74959339929 ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,...,NaN,head-liner.ru,NaN,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN
1053,Принятый,Звонок ВАТС,2023-08-23 10:15:04,79166964481,74950854280,ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,00:01:00,00:00:09,Абонент разорвал соединение,74959339929 ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,...,NaN,head-liner.ru,NaN,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN
1054,Принятый,Звонок ВАТС,2023-08-23 10:08:09,79166964481,74950854280,ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,00:00:45,00:00:09,Абонент разорвал соединение,74959339929 ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,...,NaN,head-liner.ru,NaN,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN
1055,Принятый,Звонок ВАТС,2023-08-23 09:32:28,79161079278,74951063507,Базы // Яндекс Недвижимость,00:03:20,00:00:01,Абонент разорвал соединение,Кортрос,...,NaN,head-liner.ru,NaN,Нет,NaN,NaN,NaN,г. Москва,РФ,NaN


In [ ]:
df_Headliner2.Теги.unique()

array(['Нецелевой', 'Целевой'], dtype=object)

In [ ]:
counts = df_Headliner2['Теги'].value_counts()
print(counts)

Нецелевой    767
Целевой      290
Name: Теги, dtype: int64


In [ ]:
df_Headliner2.to_csv('df_Headliner_дополнительная.csv', index = False)

#**Сбор аудиозаписей в общий набор.**

###*Павелецкая_(pavcity.turbo.site).*

In [ ]:
import pandas as pd
import os
import shutil
import zipfile

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи звонков_павелецкая сити-20231022T075749Z-001.zip'

In [ ]:
path = '/content/Записи звонков_павелецкая сити'
new_folder_path = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Общий_calls/Нецелевой'

In [ ]:
df_Pavcity_turbo_site = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Pavcity_turbo_site.csv')
filtered_df = df_Pavcity_turbo_site[df_Pavcity_turbo_site['Теги'] == 'Нецелевой']
unique_ids = filtered_df['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_path, file))
      os.remove(os.path.join(path, file))

###*Павелецкая_ЖК_«Павелецкая_сити».*

In [ ]:
import pandas as pd
import os
import shutil
import zipfile

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи звонков_павелецкая сити-20231025T094624Z-001.zip'

In [ ]:
path = '/content/Записи звонков_павелецкая сити'
new_folder_Нецелевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Общий_calls/Нецелевой'
new_folder_Целевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Общий_calls/Целевой'
df_Paveletskaya = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Paveletskaya.csv')

In [ ]:
filtered_df_Нецелевой = df_Paveletskaya[df_Paveletskaya['Теги'] == 'Нецелевой']
unique_ids = filtered_df_Нецелевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Нецелевой, file))
      os.remove(os.path.join(path, file))

In [ ]:
filtered_df_Целевой = df_Paveletskaya[df_Paveletskaya['Теги'] == 'Целевой']
unique_ids = filtered_df_Целевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Целевой, file))
      os.remove(os.path.join(path, file))

###*Дополнительный DF по Павелецкая_ЖК_«Павелецкая_сити».*

In [ ]:
import pandas as pd
import os
import shutil
import zipfile

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи_Павелецкая_сити_дополнительные-20231028T005609Z-001.zip'

In [ ]:
path = '/content/Записи Павелецкая сити'
new_folder_Нецелевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Общий_calls/Нецелевой'
new_folder_Целевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Общий_calls/Целевой'
df_Paveletskaya2 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Paveletskaya_дополнительная.csv')

In [ ]:
filtered_df_Нецелевой = df_Paveletskaya2[df_Paveletskaya2['Теги'] == 'Нецелевой']
unique_ids = filtered_df_Нецелевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Нецелевой, file))
      os.remove(os.path.join(path, file))

In [ ]:
filtered_df_Целевой = df_Paveletskaya2[df_Paveletskaya2['Теги'] == 'Целевой']
unique_ids = filtered_df_Целевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Целевой, file))
      os.remove(os.path.join(path, file))

###*Primavera.*

In [ ]:
import pandas as pd
import os
import shutil
import zipfile

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи звонков_primavera-20231026T154044Z-001.zip'

In [ ]:
path = '/content/Записи звонков_primavera'
new_folder_Нецелевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Общий_calls/Нецелевой'
new_folder_Целевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Общий_calls/Целевой'
df_Primavera = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Primavera.csv')

In [ ]:
filtered_df_Нецелевой = df_Primavera[df_Primavera['Теги'] == 'Нецелевой']
unique_ids = filtered_df_Нецелевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Нецелевой, file))
      os.remove(os.path.join(path, file))

In [ ]:
filtered_df_Целевой = df_Primavera[df_Primavera['Теги'] == 'Целевой']
unique_ids = filtered_df_Целевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Целевой, file))
      os.remove(os.path.join(path, file))

###*Дополнительный DF по Primavera.*

In [ ]:
import pandas as pd
import os
import shutil
import zipfile

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи_Primavera_дополнительные-20231028T005838Z-001.zip'

In [ ]:
path = '/content/Записи Primavera'
new_folder_Нецелевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Общий_calls/Нецелевой'
new_folder_Целевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Общий_calls/Целевой'
df_Primavera2 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Primavera_дополнительная.csv')

In [ ]:
filtered_df_Нецелевой = df_Primavera2[df_Primavera2['Теги'] == 'Нецелевой']
unique_ids = filtered_df_Нецелевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Нецелевой, file))
      os.remove(os.path.join(path, file))

In [ ]:
filtered_df_Целевой = df_Primavera2[df_Primavera2['Теги'] == 'Целевой']
unique_ids = filtered_df_Целевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Целевой, file))
      os.remove(os.path.join(path, file))

###*Headliner.*

In [ ]:
import pandas as pd
import os
import shutil
import zipfile

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи звонков_хедлайнер-20231026T173308Z-001.zip'

In [ ]:
path = '/content/Записи звонков_хедлайнер'
file_dict = {}
for file in os.listdir(path):
  unique_number = file.split('_')[7]
  if unique_number in file_dict:
    print(f'Files {file_dict[unique_number]} and {file} have the same unique number: {unique_number}')
    os.remove(os.path.join(path, file))
  else:
    file_dict[unique_number] = file

Files 2023-06-10_12-51-20.873488_from_74957457444_to_74959339929_session_2996988633_talk.mp3 and 2023-06-10_12-51-20.873488_from_74957457444_to_74959339929_session_2996988633_talk(1).mp3 have the same unique number: 2996988633
Files 2023-06-10_12-50-13.808032_from_79057912134_to_74997552563_session_2996260363_talk(1).mp3 and 2023-06-10_12-50-13.808032_from_79057912134_to_74997552563_session_2996260363_talk.mp3 have the same unique number: 2996260363
Files 2023-06-10_12-20-01.703751_from_79855320219_to_74959339902_session_2996196943_talk(1).mp3 and 2023-06-10_12-20-01.703751_from_79855320219_to_74959339902_session_2996196943_talk.mp3 have the same unique number: 2996196943
Files 2023-06-10_17-09-27.282329_from_79636310102_to_74997552563_session_2979780790_talk.mp3 and 2023-06-10_17-09-27.282329_from_79636310102_to_74997552563_session_2979780790_talk(1).mp3 have the same unique number: 2979780790
Files 2023-06-10_12-41-39.438371_from_79932311337_to_74997552563_session_3014808800_talk.mp3

In [ ]:
path = '/content/Записи звонков_хедлайнер'
new_folder_Нецелевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Общий_calls/Нецелевой'
new_folder_Целевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Общий_calls/Целевой'
df_Headliner = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Headliner.csv')

In [ ]:
filtered_df_Нецелевой = df_Headliner[df_Headliner['Теги'] == 'Нецелевой']
unique_ids = filtered_df_Нецелевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Нецелевой, file))
      os.remove(os.path.join(path, file))

In [ ]:
filtered_df_Целевой = df_Headliner[df_Headliner['Теги'] == 'Целевой']
unique_ids = filtered_df_Целевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Целевой, file))
      os.remove(os.path.join(path, file))

###*Дополнительный DF по Headliner.*

In [ ]:
import pandas as pd
import os
import shutil
import zipfile

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи_Headliner_дополнительные-20231028T005642Z-001.zip'

In [ ]:
path = '/content/Записи Headliner'
new_folder_Нецелевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Общий_calls/Нецелевой'
new_folder_Целевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Общий_calls/Целевой'
df_Headliner2 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Headliner_дополнительная.csv')

In [ ]:
filtered_df_Нецелевой = df_Headliner2[df_Headliner2['Теги'] == 'Нецелевой']
unique_ids = filtered_df_Нецелевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Нецелевой, file))
      os.remove(os.path.join(path, file))

In [ ]:
filtered_df_Целевой = df_Headliner2[df_Headliner2['Теги'] == 'Целевой']
unique_ids = filtered_df_Целевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Целевой, file))
      os.remove(os.path.join(path, file))

#**Сбор аудиозаписей в тестовый набор.**

###*Павелецкая_ЖК_«Павелецкая_сити».*

In [ ]:
import pandas as pd
import os
import shutil
import zipfile

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи звонков_павелецкая сити-20231025T094624Z-001.zip'

In [ ]:
path = '/content/Записи звонков_павелецкая сити'
new_folder_Нецелевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Тестовый_calls/Нецелевой'
new_folder_Целевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Тестовый_calls/Целевой'
df_Paveletskaya = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Test_Pav1.csv')

In [ ]:
filtered_df_Нецелевой = df_Paveletskaya[df_Paveletskaya['Теги'] == 'Нецелевой']
unique_ids = filtered_df_Нецелевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Нецелевой, file))
      os.remove(os.path.join(path, file))

In [ ]:
filtered_df_Целевой = df_Paveletskaya[df_Paveletskaya['Теги'] == 'Целевой']
unique_ids = filtered_df_Целевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Целевой, file))
      os.remove(os.path.join(path, file))

###*Дополнительный DF по Павелецкая_ЖК_«Павелецкая_сити».*

In [ ]:
import pandas as pd
import os
import shutil
import zipfile

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи_Павелецкая_сити_дополнительные-20231028T005609Z-001.zip'

In [ ]:
path = '/content/Записи Павелецкая сити'
new_folder_Нецелевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Тестовый_calls/Нецелевой'
new_folder_Целевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Тестовый_calls/Целевой'
df_Paveletskaya2 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Test_Pav2.csv')

In [ ]:
filtered_df_Нецелевой = df_Paveletskaya2[df_Paveletskaya2['Теги'] == 'Нецелевой']
unique_ids = filtered_df_Нецелевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Нецелевой, file))
      os.remove(os.path.join(path, file))

In [ ]:
filtered_df_Целевой = df_Paveletskaya2[df_Paveletskaya2['Теги'] == 'Целевой']
unique_ids = filtered_df_Целевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Целевой, file))
      os.remove(os.path.join(path, file))

###*Primavera.*

In [ ]:
import pandas as pd
import os
import shutil
import zipfile

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи звонков_primavera-20231026T154044Z-001.zip'

In [ ]:
path = '/content/Записи звонков_primavera'
new_folder_Нецелевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Тестовый_calls/Нецелевой'
new_folder_Целевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Тестовый_calls/Целевой'
df_Primavera = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Test_Prim1.csv')

In [ ]:
filtered_df_Нецелевой = df_Primavera[df_Primavera['Теги'] == 'Нецелевой']
unique_ids = filtered_df_Нецелевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Нецелевой, file))
      os.remove(os.path.join(path, file))

In [ ]:
filtered_df_Целевой = df_Primavera[df_Primavera['Теги'] == 'Целевой']
unique_ids = filtered_df_Целевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Целевой, file))
      os.remove(os.path.join(path, file))

###*Дополнительный DF по Primavera.*

In [ ]:
import pandas as pd
import os
import shutil
import zipfile

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи_Primavera_дополнительные-20231028T005838Z-001.zip'

In [ ]:
path = '/content/Записи Primavera'
new_folder_Нецелевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Тестовый_calls/Нецелевой'
new_folder_Целевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Тестовый_calls/Целевой'
df_Primavera2 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Test_Prim2.csv')

In [ ]:
filtered_df_Нецелевой = df_Primavera2[df_Primavera2['Теги'] == 'Нецелевой']
unique_ids = filtered_df_Нецелевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Нецелевой, file))
      os.remove(os.path.join(path, file))

In [ ]:
filtered_df_Целевой = df_Primavera2[df_Primavera2['Теги'] == 'Целевой']
unique_ids = filtered_df_Целевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Целевой, file))
      os.remove(os.path.join(path, file))

###*Headliner.*

In [ ]:
import pandas as pd
import os
import shutil
import zipfile

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи звонков_хедлайнер-20231026T173308Z-001.zip'

In [ ]:
path = '/content/Записи звонков_хедлайнер'
file_dict = {}
for file in os.listdir(path):
  unique_number = file.split('_')[7]
  if unique_number in file_dict:
    print(f'Files {file_dict[unique_number]} and {file} have the same unique number: {unique_number}')
    os.remove(os.path.join(path, file))
  else:
    file_dict[unique_number] = file

In [ ]:
path = '/content/Записи звонков_хедлайнер'
new_folder_Нецелевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Тестовый_calls/Нецелевой'
new_folder_Целевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Тестовый_calls/Целевой'
df_Headliner = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Test_Head1.csv')

In [ ]:
filtered_df_Нецелевой = df_Headliner[df_Headliner['Теги'] == 'Нецелевой']
unique_ids = filtered_df_Нецелевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Нецелевой, file))
      os.remove(os.path.join(path, file))

In [ ]:
filtered_df_Целевой = df_Headliner[df_Headliner['Теги'] == 'Целевой']
unique_ids = filtered_df_Целевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Целевой, file))
      os.remove(os.path.join(path, file))

###*Дополнительный DF по Headliner.*

In [ ]:
import pandas as pd
import os
import shutil
import zipfile

In [ ]:
!unzip -qo '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Записи_Headliner_дополнительные-20231028T005642Z-001.zip'

In [ ]:
path = '/content/Записи Headliner'
new_folder_Нецелевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Тестовый_calls/Нецелевой'
new_folder_Целевой = '/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Тестовый_calls/Целевой'
df_Headliner2 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Test_Head2.csv')

In [ ]:
filtered_df_Нецелевой = df_Headliner2[df_Headliner2['Теги'] == 'Нецелевой']
unique_ids = filtered_df_Нецелевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Нецелевой, file))
      os.remove(os.path.join(path, file))

In [ ]:
filtered_df_Целевой = df_Headliner2[df_Headliner2['Теги'] == 'Целевой']
unique_ids = filtered_df_Целевой['Идентификатор сессии звонка'].unique()
for file in os.listdir(path):
  for id in unique_ids:
    if str(id) in file:
      shutil.copy2(os.path.join(path, file), os.path.join(new_folder_Целевой, file))
      os.remove(os.path.join(path, file))

#**Сбор df_Common и df_Test.**

##df_Common.

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Pavcity_turbo_site.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Paveletskaya.csv')
df3 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Paveletskaya_дополнительная.csv')
df4 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Primavera.csv')
df5 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Primavera_дополнительная.csv')
df6 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Headliner.csv')
df7 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Headliner_дополнительная.csv')
df_Common = pd.concat([df1, df2, df3, df4, df5, df6, df7])

In [ ]:
df_Common = df_Common.reset_index(drop = True)

In [ ]:
df_Common

,Статус,Тип,Дата и время,Номер абонента,Длительность звонка,Длительность ожидания ответа,Сотрудник,ID посетителя,Номер обращения,Теги,...,Расширенная UTM-метка Medium,Расширенная UTM-метка Content,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 72,Unnamed: 73
0,Принятый,Динамический коллтрекинг,2023-07-18 13:51:58,7.800506e+10,00:00:02,00:00:01,74955141111.0,7.046435e+09,2.0,Нецелевой,...,cpc,v2||64424254||11012716629||33325751427||купить...,купить двушку на юге москвы,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Принятый,Динамический коллтрекинг,2023-07-11 18:13:13,7.927415e+10,00:00:04,00:00:01,74955141111.0,7.098722e+09,2.0,Нецелевой,...,cpc,v2||64424254||11012716629||44050215065||---aut...,---autotargeting,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Принятый,Аналитика,2023-06-27 15:04:42,7.903618e+10,00:00:22,00:00:02,74955141111.0,NaN,1.0,Нецелевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Принятый,Динамический коллтрекинг,2023-06-19 20:04:38,7.927457e+10,00:00:04,00:00:01,74955141111.0,7.371932e+09,1.0,Нецелевой,...,cpc,v2||64424254||11012716635||44050215065||---aut...,---autotargeting,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Принятый,Аналитика,2023-07-20 11:43:08,7.916646e+10,00:07:09,00:00:01,SIP URI,NaN,1.0,Целевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5954,Принятый,Звонок ВАТС,2023-08-23 10:20:52,7.916696e+10,00:00:55,00:00:10,ОКС,NaN,4.0,Нецелевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5955,Принятый,Звонок ВАТС,2023-08-23 10:15:04,7.916696e+10,00:01:00,00:00:09,ОКС,NaN,3.0,Нецелевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5956,Принятый,Звонок ВАТС,2023-08-23 10:08:09,7.916696e+10,00:00:45,00:00:09,ОКС,NaN,2.0,Нецелевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5957,Принятый,Звонок ВАТС,2023-08-23 09:32:28,7.916108e+10,00:03:20,00:00:01,Кортрос,7.506091e+09,12.0,Нецелевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
counts = df_Common['Теги'].value_counts()
print(counts)

Нецелевой    3687
Целевой      2272
Name: Теги, dtype: int64


In [ ]:
df_Common.to_csv('df_Common.csv', index = False)

##df_Test.

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Test_Pav1.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Test_Pav2.csv')
df3 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Test_Prim1.csv')
df4 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Test_Prim2.csv')
df5 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Test_Head1.csv')
df6 = pd.read_csv('/content/drive/MyDrive/Классификация_аудиозвонков_на_целевые_и_нецелевые/Датасеты обработанные/df_Test_Head2.csv')
df_Test = pd.concat([df1, df2, df3, df4, df5, df6])

In [ ]:
df_Test = df_Test.reset_index(drop = True)

In [ ]:
df_Test

,Статус,Тип,Дата и время,Номер абонента,Длительность ожидания ответа,Сотрудник,Рекламная кампания,Номер обращения,Длительность звонка,Теги,...,Расширенная UTM-метка Medium,Расширенная UTM-метка Content,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 72,Unnamed: 73
0,Принятый,Аналитика,2023-07-12 12:40:34,7.960588e+10,00:00:01,SIP URI,Media108 | Павелецкая Сити | xml | cian.ru,1.0,00:06:25,Целевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Принятый,Аналитика,2023-06-26 15:20:22,7.920771e+10,00:00:02,74955141111,MRGroup | Павелецкая Сити | sms | СМС рассылка...,9.0,00:03:43,Целевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Принятый,Аналитика,2023-06-28 15:33:39,7.903203e+10,00:00:01,SIP URI,Media108 | Павелецкая Сити | media | Realty.ru,1.0,00:07:07,Целевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Принятый,Аналитика,2023-07-02 11:16:33,7.916486e+10,00:00:01,SIP URI,Media108 | Павелецкая Сити | xml | Яндекс.Недв...,1.0,00:08:31,Целевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Принятый,Аналитика,2023-07-18 15:49:21,7.916435e+10,00:00:01,SIP URI,Media108 | Павелецкая Сити | xml | cian.ru,1.0,00:06:29,Целевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,Принятый,Динамический коллтрекинг,2023-10-02 19:29:52,7.925500e+10,00:00:05,Кортрос,Поисковое продвижение,23.0,00:02:17,Нецелевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
658,Принятый,Звонок ВАТС,2023-09-05 17:57:46,7.926932e+10,00:00:10,ОКС,ОТДЕЛ КЛИЕНТСКОГО СЕРВИСА Хедлайнер,1.0,00:01:25,Нецелевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
659,Принятый,Звонок ВАТС,2023-09-08 16:09:01,7.958414e+10,00:00:00,Кортрос,Социальные сети / VK,1.0,00:00:02,Нецелевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
660,Принятый,Динамический коллтрекинг,2023-10-02 16:35:03,7.927945e+10,00:00:05,Кортрос,Яндекс Дисплей,2033.0,00:00:13,Нецелевой,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
counts = df_Test['Теги'].value_counts()
print(counts)

Нецелевой    409
Целевой      253
Name: Теги, dtype: int64


In [ ]:
df_Test.to_csv('df_Test.csv', index = False)